## Policy Gradient

### Reference
- Medium post by Arthur Juliani: [link](https://medium.com/@awjuliani/super-simple-reinforcement-learning-tutorial-part-2-ded33892c724#.mtwpvfi8b)
- Github for original code: [link](https://github.com/awjuliani/DeepRL-Agents)
- Andrej Karpathy Blog post: [link](https://gist.github.com/karpathy/a4166c7fe253700972fcbc77e4ea32c5)

### 1. Import Modules

In [1]:
from __future__ import absolute_import, division, print_function
from six.moves import xrange
import numpy as np
import tensorflow as tf
import os, sys, math
import matplotlib.pyplot as plt
%matplotlib inline

### Cart-Pole Problem
- Task: Balance the pole on the cart without falling it down
- State : observation in 4 tuple
    1. x: position of the cart
    2. theta: angle of the pole (default = 0, when vertical to the cart)
    3. dx/dt : velocity of the cart
    4. dtheta/dt : rate of change of the angle
- Action: [Left, Right]
- Reward: +1 for every time step (no negative reward -> need for reward standardization)
- Goal: Reach for 200 score (stay balanced for 200 time frames)
- 0.000001 change in state may result in huge Q-value difference
- Need for flexible network to approximate the solution

In [2]:
import gym
env = gym.make('CartPole-v0')

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


#### Necessary code for game environment
1. env.reset(): reset the game environment to default position, restart episode
2. env.render(): show visualiztion of current state
3. next\_state, reward, done, _ = env.step(action)
    - next_state: next state after given action is taken 
    - reward: reward from the action, here all the reward is set to 1 
    - done: if episode is ended. if done == True, break the episode loop

In [6]:
# simulate environment with sample actions

for i_episode in range(2):
    state = env.reset()
    for t in range(20):
        env.render()
        #print(state)
        # sample random action from available action list
        action = env.action_space.sample()
        next_state, reward, done, _ = env.step(action)
        #print(reward)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break

Episode finished after 16 timesteps


### 2. Graphing

#### 1) Set up hyperparameters

#### 2) Forward Propagation
- Input: Current state as 1x4 tensor / Output: probability to take action "right"
- Neural Network with 1 hidden layer: total 2 weights and 2 biass to be trained
- Logit will be treated as "score" for a certain action

#### 3) Back Propagation
- Labels: actions taken (1 for right, 0 for left)
- Logits: final output from neural network
- Log-likelihood error: here, we use cross entropy method instead
- Advantage: discounted cumulative future reward of given action
- Loss: log-likelihood * Advantage
- Advantage will guide the gradient toward or away from optimum
- Optimizer: optimize with decaying learning rate, here we use Adam

#### 4) Gradient Update
- For stabilized learning, gradient will be updated slowly

In [3]:
## hyperparameters ##

input_dim = 4 # input dimensionality
hidden_dim = 10 # number of hidden layer neurons
batch_size = 5 # every how many episodes to do a param update?
learning_rate = 1e-2 # feel free to play with this to train faster or more stably.
gamma = 0.99 # discount factor for rewar
total_episodes = 10000 # the number of episodes
logdir = "./ce_log" # directory for storing summaries and checkpoints

In [4]:
## Graph ##

main_graph = tf.Graph()
with main_graph.as_default():
    
    #This defines the network as it goes from taking an observation of the environment to 
    #giving a probability of chosing to the action of moving left or right
    
    with tf.name_scope("Variables"):
        
        # for forward-propagation #
        
        # input_x will be current state containing 4 elements each
        input_x = tf.placeholder(dtype=tf.float32, shape=[None, input_dim] , name="input_x")
        
        xavier_init = tf.contrib.layers.xavier_initializer()
        
        # Neural network with 1 hidden layer
        W1 = tf.Variable(name="input_hidden_W", initial_value=xavier_init([input_dim, hidden_dim]), 
                         dtype=tf.float32)
        b1 = tf.Variable(name="input_hidden_b", initial_value=xavier_init([hidden_dim]), dtype=tf.float32)
        
        W2 = tf.Variable(name="hidden_output_W", initial_value=xavier_init([hidden_dim, 1]), dtype=tf.float32)
        b2 = tf.Variable(name="hidden_output_b", initial_value=xavier_init([1]), dtype=tf.float32)
        
        # for back-propagation #
        
        # input_y will be actions from predicted policy
        # advantages will be discounted reward sum of each state
        # advantage will guide the network as a multiplier for applying gradients
        input_y = tf.placeholder(dtype=tf.float32, shape=[None,1], name="input_y")
        advantages = tf.placeholder(dtype=tf.float32, name="reward_signal")
        
        # Gradient Buffers #
        
        # Placeholders to send the final gradients through when we update.
        W1Grad = tf.placeholder(dtype=tf.float32,name="batch_grad_W1") 
        W2Grad = tf.placeholder(dtype=tf.float32,name="batch_grad_W2")
        b1Grad = tf.placeholder(dtype=tf.float32,name="batch_grad_b1")
        b2Grad = tf.placeholder(dtype=tf.float32,name="batch_grad_b2")
        
        # Tensorboard summary
        tf.summary.histogram("Input_to_hidden_Weight", W1)
        tf.summary.histogram("Input_to_hidden_bias", b1)
        tf.summary.histogram("Hidden_to_output_Weight", W2)
        tf.summary.histogram("Hidden_to_output_bias", b2)
        
    with tf.name_scope("forward_propagation"):
        # Dimension Changes:
        # input: [batch_size, 4] -> hidden: [batch_size, 10] -> output: [batch_size, 1]
        
        # first hidden layer utilizes ReLU activation
        layer1 = tf.nn.relu(tf.add(tf.matmul(input_x, W1), b1))
        
        # we call final logit "score", it measures score for choosing certain action
        # the whole network approximates scoring function.
        score = tf.add(tf.matmul(layer1,W2), b2, name="score")
        
        # sigmoid function as actiavtion as output type is binary (left or right)
        # single number between 0 or 1 will be probability to go "Right"
        probability = tf.nn.sigmoid(score)
        
        # Tensorboard summary
        tf.summary.histogram("Score", score)
    
    with tf.name_scope("back_propagation"):
        
        # get the variables to be updated
        tvars = tf.trainable_variables()
        
        # The loss function. This sends the weights in the direction of making actions 
        # that gave good advantage (reward over time) more likely, and actions that didn't less likely.
        
        # calculate loss between action taken (will be 1 or 0 like label) and score (logit from Neural Network) 
        loglik = tf.nn.sigmoid_cross_entropy_with_logits(labels=input_y, logits=score, name="cross_entropy")
        
        # loss is multiplied with advantages to determine direction for optimization
        # if advantage is positive, optimize by minimizing the loss
        # elif advantage is negative, optimize by maximizing the loss
        loss = tf.reduce_sum(loglik * advantages, name="Loss") 
        
        loglik2 = (input_y)*(-tf.log(input_y - probability)) + (1-input_y)*(-tf.log(input_y + probability))
        #loss = -tf.reduce_mean(loglik * advantages) 
        
        # calculate gradient of trainable variables
        newGrads = tf.gradients(loss,tvars, name="Gradients")
        
        tf.summary.scalar("Loss", loss)
        
    with tf.name_scope("Gradient_update"):
        
        # Once we have collected a series of gradients from multiple episodes, we apply them.
        # We don't just apply gradeients after every episode in order to account for noise in the reward signal.
                
        # merge gradients for update
        batchGrad = [W1Grad,b1Grad,W2Grad,b2Grad]
        
        # Our optimizer
        # Adam or RMSProp is preferred (Optimizer with decaying learning rate is recommended)
        adam = tf.train.AdamOptimizer(learning_rate=learning_rate) 
        
        # update gradient value
        # apply gradients slowly
        updateGrads = adam.apply_gradients(zip(batchGrad,tvars), name="update")
        
    with tf.name_scope("utils"):
        
        # make log directory
        if not os.path.exists(logdir):
            os.mkdir(logdir)
        
        # writer and saver for model
        saver = tf.train.Saver()
        writer = tf.summary.FileWriter(logdir=logdir)
        summary = tf.summary.merge_all()
        
print("Graphing Done!")

Graphing Done!


#### Calculating Advantage
- get all the reward value
- sum up rewards with discount factor

In [5]:
def discount_rewards(r):
    """ take 1D float array of rewards and compute discounted reward """
    # initialize with zeros
    discounted_r = np.zeros_like(r)
    # set start value
    running_add = 0
    
    # add up gamma*r[t] for each step in episode
    for t in reversed(xrange(0, r.size)):
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
        
    return discounted_r

### 3. Train
- load variables if checkpoint exists
- stack state, action and reward into buffers
- if episode is done, start the training based on stacked info
- advantage will be discounted future reward calculated from reward buffer
- Loss & Gradients computation
- Update gradients slowly
- Write summary and checkpoints
- if average score exceeds 190, episodes will be visualized
- if average score reaches 200, training will be finished

In [67]:
# set necessary arrays and values
state_buffer, reward_buffer, action_buffer = [],[],[]
running_reward = None
reward_sum = 0
episode_number = 1

# Launch the graph
with tf.Session(graph=main_graph) as sess:

    # check for TensorFlow checkpoint
    # if exists, restore Variables
    ckpt = tf.train.get_checkpoint_state(logdir)
    
    if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
        print("Reading model parameters...")
        saver.restore(sess, ckpt.model_checkpoint_path)
        
    else:
        print("Initializing all variables...")
        sess.run(tf.global_variables_initializer())
    
    # add graph to summary
    writer.add_graph(sess.graph)
    
    # we will not see the graphics during training
    rendering = False
    
    # Obtain an initial observation of the environment
    state = env.reset() 

    # Reset the gradient placeholder. We will collect gradients in 
    # gradBuffer until we are ready to update our policy network. 
    gradBuffer = sess.run(tvars)

    # initialize Gradient buffer by filling with 0
    for ix,grad in enumerate(gradBuffer):
        gradBuffer[ix] = grad * 0
    
    while episode_number <= total_episodes:
        
        # Rendering the environment slows things down, 
        # so let's only look at it once our agent is doing a good job.
        
        if reward_sum / batch_size > 190 or rendering == True : 
            env.render()
            rendering = True
            
        # Make sure the observation is in a shape the network can handle.
        state_input = np.reshape(state, [1, input_dim])
        
        # Run the policy network and get an action to take. 
        curr_policy = sess.run(probability, feed_dict={input_x:state_input})
        
        # get the action from predicted policy
        action = 1 if np.random.uniform() < curr_policy else 0
        #y = 0 if action==1 else 1
        y = action
        
        # add state info
        state_buffer.append(state_input)
        
        # add action info
        action_buffer.append(y)

        # step the environment and get new measurements
        next_state, reward, done, _ = env.step(action)
        
        # move to next state
        state = next_state
        
        # add up reward
        reward_sum += reward
        
        # record reward 
        reward_buffer.append(reward) 
        
        # if episode is finished
        if done:
            # add episode
            episode_number += 1
            
            # stack together all inputs, actions and reward for each episode
            episode_states = np.vstack(state_buffer)
            episode_actions = np.vstack(action_buffer)
            episode_rewards = np.vstack(reward_buffer)
            
            # reset array memory
            state_buffer, reward_buffer, action_buffer = [],[],[] 

            # compute the discounted reward backwards through time
            discounted_reward_ep = discount_rewards(episode_rewards)
            
            # size the rewards to be unit normal (helps control the gradient estimator variance)
            # first half would be Good actions (pos) while latter hald would be Bad actions (neg)
            discounted_reward_ep -= np.mean(discounted_reward_ep)
            discounted_reward_ep //= np.std(discounted_reward_ep)
            
            # Get the gradient for this episode, and save it in the gradBuffer
            # x: states / y: actions / Advantage: discounted reward
            tGrad = sess.run(newGrads,feed_dict={input_x: episode_states, input_y: episode_actions, 
                                                 advantages: discounted_reward_ep})
            
            # save into buffer, do not update right away
            # prevent unstable learning
            for ix,grad in enumerate(tGrad):
                gradBuffer[ix] += grad
                
            # If we have completed enough episodes, then update the policy network with our gradients.
            # assgin values from buffer
            if episode_number % batch_size == 0: 
                grad_buffer_feeddict = {W1Grad: gradBuffer[0],b1Grad:gradBuffer[1], 
                                        W2Grad: gradBuffer[2],b2Grad: gradBuffer[3]}
                
                sess.run(updateGrads,feed_dict=grad_buffer_feeddict)
                
                # reset grad buffer to 0
                for ix,grad in enumerate(gradBuffer):
                    gradBuffer[ix] = grad * 0
                
                ## Result printing
                # Give a summary of how well our network is doing for each batch of episodes.
                running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
                print('Average reward for episode {:.2f}.  Total average reward {:.2f}.' 
                      .format(reward_sum//batch_size, running_reward//batch_size))
                
                if reward_sum//batch_size >= 200: 
                    print("Task solved in",episode_number,'episodes!')
                    break
                    
                reward_sum = 0
                
            if episode_number % (5*batch_size) == 0: 
                saver.save(sess, os.path.join(logdir, "model.ckpt"), global_step=episode_number)
                curr_summary = sess.run(summary, feed_dict={input_x: episode_states, input_y: episode_actions, 
                                                 advantages: discounted_reward_ep})
                curr_loss, curr_loss_1 = sess.run([loglik, loglik2], feed_dict={input_x: episode_states, input_y: episode_actions, 
                                                 advantages: discounted_reward_ep})
                print("ce:\n")
                print(curr_loss)
                print("ll:\n")
                print(curr_loss_1)
                writer.add_summary(curr_summary, global_step=episode_number)
                print("Model saved!")
                print("Summary written!")
            
            state = env.reset()
        
print(episode_number,'Episodes completed.')

Reading model parameters...
INFO:tensorflow:Restoring parameters from ./ce_log/model.ckpt-250
Average reward for episode 108.00.  Total average reward 108.00.
Average reward for episode 121.00.  Total average reward 108.00.
Average reward for episode 184.00.  Total average reward 109.00.
Average reward for episode 157.00.  Total average reward 109.00.
Average reward for episode 103.00.  Total average reward 109.00.
ce:

[[0.84547013]
 [0.3197832 ]
 [0.52871746]
 [0.55265975]
 [0.9205006 ]
 [0.3142205 ]
 [0.47200522]
 [0.76931536]
 [0.35048634]
 [0.6368471 ]
 [0.43424833]
 [0.66116554]
 [0.4147884 ]
 [0.6870347 ]
 [0.3959471 ]
 [0.7148518 ]
 [0.37898675]
 [0.74426216]
 [1.1929665 ]
 [0.29731312]
 [0.35012585]
 [0.845583  ]
 [0.34541193]
 [0.5190128 ]
 [0.55641127]
 [0.49308026]
 [0.5886071 ]
 [0.4719306 ]
 [0.77270555]
 [0.3475607 ]
 [0.7604523 ]
 [1.2186682 ]
 [0.17886172]
 [0.33420044]
 [0.804722  ]
 [0.3169293 ]
 [0.5596904 ]
 [0.52062374]
 [0.5336765 ]
 [0.86519724]
 [0.34094226]
 [

Average reward for episode 123.00.  Total average reward 112.00.
Average reward for episode 143.00.  Total average reward 112.00.
Average reward for episode 177.00.  Total average reward 113.00.
Average reward for episode 184.00.  Total average reward 114.00.
Average reward for episode 134.00.  Total average reward 114.00.
ce:

[[0.71016055]
 [0.32665202]
 [0.64166284]
 [0.42606977]
 [0.62786865]
 [0.4331893 ]
 [0.6193984 ]
 [0.4369817 ]
 [0.6158227 ]
 [1.0374438 ]
 [0.21865831]
 [0.41570735]
 [0.6572789 ]
 [0.40118203]
 [0.6830585 ]
 [0.38348252]
 [0.7161209 ]
 [0.36262396]
 [0.7578927 ]
 [0.33867857]
 [0.59153056]
 [1.0676621 ]
 [0.26547658]
 [0.41988176]
 [0.81663764]
 [0.31006953]
 [0.5556882 ]
 [1.0101191 ]
 [0.26982105]
 [0.44328988]
 [0.84196824]
 [0.2988909 ]
 [0.869159  ]
 [0.28422835]
 [0.9084532 ]
 [0.26521578]
 [0.48168173]
 [0.5373865 ]
 [0.4609708 ]
 [0.5642125 ]
 [0.4408855 ]
 [0.8053514 ]
 [0.299138  ]
 [0.7996396 ]
 [0.29703617]
 [0.8076976 ]
 [1.3713989 ]
 [0.20931664

Average reward for episode 171.00.  Total average reward 117.00.
Average reward for episode 160.00.  Total average reward 118.00.
Average reward for episode 119.00.  Total average reward 118.00.
Average reward for episode 184.00.  Total average reward 118.00.
Average reward for episode 177.00.  Total average reward 119.00.


KeyboardInterrupt: 